In [ ]:
!pip install ninja
!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html



 

     |████████████████████████████████| 108 kB 13.8 MB/s 
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.3 MB/s eta 0:12:19tcmalloc: large alloc 1147494400 bytes == 0x559250518000 @  0x7f487eff3615 0x5592173443bc 0x55921742518a 0x5592173471cd 0x559217439b3d 0x5592173bb458 0x5592173b602f 0x559217348aba 0x5592173bb2c0 0x5592173b602f 0x559217348aba 0x5592173b7cd4 0x55921743a986 0x5592173b7350 0x55921743a986 0x5592173b7350 0x55921743a986 0x5592173b7350 0x559217348f19 0x55921738ca79 0x559217347b32 0x5592173bb1dd 0x5592173b602f 0x559217348aba 0x5592173b7cd4 0x5592173b602f 0x559217348aba 0x5592173b6eae 0x5592173489da 0x5592173b7108 0x5592173b602f
     |██████████████████▌             | 1055.7 MB 1.2 MB/s eta 0:10:28tcmalloc: large alloc 1434370048 bytes == 0x559294b6e000 @  0x7f487eff3615 0x5592173443bc 0x55921742518a 0x5592173471cd 0x559217439b3d 0x5592173bb458 0x5592173b602f 0x559217348aba 0x5592173bb2c0 0x559217

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import os
from argparse import Namespace

import numpy as np
import torch
import sys
import dlib

sys.path.append(".")
sys.path.append("..")
sys.path.append('/content/drive/MyDrive/PSPNet')
from options.test_options import TestOptions
from models.psp import pSp
import cv2
from scripts.align_all_parallel import align_face
import torchvision.transforms as transforms
import moviepy.editor as mpyeditor

# test_opts = TestOptions().parse()

test_opts = Namespace(checkpoint_path=None, couple_outputs=False, data_path='gt_images', exp_dir=None, latent_mask=None,
                      mix_alpha=None, n_images=None, n_outputs_to_generate=5, resize_factors=None, resize_outputs=False,
                      test_batch_size=2, test_workers=2)

test_opts.checkpoint_path = '/content/drive/MyDrive/PSPNet/pretrained_models/psp_ffhq_encode.pt'

# update test options with options used during training
ckpt = torch.load(test_opts.checkpoint_path, map_location='cpu')
opts = ckpt['opts']
opts.update(vars(test_opts))
opts['output_size'] = 1024

opts = Namespace(**opts)

net = pSp(opts)
net.eval()
net.cuda()
# Prepare the PSPNet + StyleGAN network

transform = transforms.Compose([
    transforms.Resize(size=(256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])]
)
# Preprocessing operations for each image

predictor = dlib.shape_predictor("/content/drive/MyDrive/PSPNet/shape_predictor_68_face_landmarks.dat")
# Predict facial landmarks

aligned_image_first = align_face(filepath="/content/drive/MyDrive/PSPNet/first.jpg", predictor=predictor)
aligned_image_first = aligned_image_first.convert("RGB")

aligned_image_last = align_face(filepath="/content/drive/MyDrive/PSPNet/last.jpg", predictor=predictor)
aligned_image_last = aligned_image_last.convert("RGB")
# Align face using the landmarks

from_im_first = transform(aligned_image_first).unsqueeze(0)
from_im_last = transform(aligned_image_last).unsqueeze(0)
# Do preprocessing on the aligned face

frame_list = []
with torch.no_grad():
    input1 = from_im_first.float().cuda()
    input2 = from_im_last.float().cuda()
    _, first = net(input1, randomize_noise=False, resize=False, return_latents=True)
    _, last = net(input2, randomize_noise=False, resize=False, return_latents=True)
    for i in range(201):
        curr = first + (last - first) * i / 200
        # Obtain features from input image

        result, _ = net.decoder([curr.float()], input_is_latent=True, randomize_noise=False, return_latents=False)
        # Feed features to the StyleGAN

        result = result.squeeze().permute((1, 2, 0)).cpu().numpy()

        result[result > 1] = 1
        result[result < -1] = -1
        result = (255 * (result + 1) // 2).astype(np.uint8)

        frame_list.append(result)

clip = mpyeditor.ImageSequenceClip(frame_list, fps=25)
clip.write_videofile('/content/drive/MyDrive/part2.mp4', codec='libx264')

Loading pSp from checkpoint: /content/drive/MyDrive/PSPNet/pretrained_models/psp_ffhq_encode.pt
[MoviePy] >>>> Building video /content/drive/MyDrive/part2.mp4
[MoviePy] Writing video /content/drive/MyDrive/part2.mp4


100%|██████████| 202/202 [00:11<00:00, 18.21it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/MyDrive/part2.mp4 

